In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../pruebillas.ipynb'), '..'))
os.chdir(root_dir)

sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, plot_heatmaps, resample_apply, barssince
import pandas as pd
import talib as ta
import pandas_ta as pandas_ta
import numpy as np
import matplotlib.pyplot as plt
from backtesting import Strategy
import plotly.express as px

from datetime import datetime
import MetaTrader5 as mt5

import random
random.seed(42)

from backbone.utils.general_purpose import diff_pips

In [3]:
def plot_stats(data, stats, strategy, plot=False):
    equity_curve = stats._equity_curve
    aligned_data = data.reindex(equity_curve.index)
    bt = Backtest(aligned_data, strategy, cash=15_000, commission=0.002)
    print(stats)
    if plot:
        bt.plot(results=stats, resample=False)

In [4]:
def plot_full_equity_curve(data, stats_list, warmup_bars, lookback_bars, overlay_price=True):
    equity_curves = [x["_equity_curve"].iloc[warmup_bars:] for x in stats_list]

    combined = pd.Series(dtype=float)
    for curve in equity_curves:
        # normalized_curve = curve["Equity"] / curve["Equity"].iloc[0]  # Normaliza la curva a su valor inicial
        if combined.empty:
            combined = curve["Equity"]
        else:
            # Alinea la nueva curva con la última de la serie combinada
            # normalized_curve = normalized_curve * combined.iloc[-1]
            combined = pd.concat([combined, curve["Equity"]])

    fig = px.line(x=combined.index, y=combined)
    fig.update_traces(textposition="bottom right")
    fig.show()


In [14]:
symbols_path = './backbone/data/backtest/symbols/USDCAD.csv'
df = pd.read_csv(symbols_path)
df
# print("MetaTrader5 package author: ", mt5.__author__)
# print("MetaTrader5 package version: ", mt5.__version__)

# # establish connection to MetaTrader 5 terminal
# if not mt5.initialize():
#     raise Exception("initialize() failed, error code =",mt5.last_error())

# # set time zone to UTC
# timezone = pytz.timezone("Etc/UTC")

# # create 'datetime' objects in UTC time zone to avoid the implementation of a local time zone offset
# utc_from = datetime(2024, 6, 1, tzinfo=timezone)
# utc_to = datetime(2024, 8, 1, tzinfo=timezone)
# rates = mt5.copy_rates_range('USDCAD', mt5.TIMEFRAME_M1
# , utc_from, utc_to)

# mt5.shutdown()

# # create DataFrame out of the obtained data
# df = pd.DataFrame(rates)

# # convert time in seconds into the datetime format
# df['time'] = pd.to_datetime(df['time'], unit='s')
                          
# df = df.rename(columns={
#   'time':'Date', 
#   'open':'Open', 
#   'high':'High', 
#   'low':'Low', 
#   'close':'Close', 
#   'tick_volume':'Volume'
# })

# df

,Date,Open,High,Low,Close,Volume
0,2021-08-02 00:00:00,1.24668,1.24721,1.24668,1.24719,118.27
1,2021-08-02 00:01:00,1.24722,1.24726,1.24709,1.24725,45.03
2,2021-08-02 00:02:00,1.24721,1.24721,1.24694,1.24713,87.82
3,2021-08-02 00:03:00,1.24714,1.24743,1.24713,1.24741,61.90
4,2021-08-02 00:04:00,1.24739,1.24739,1.24706,1.24720,94.39
...,...,...,...,...,...,...
897525,2024-01-01 23:55:00,1.32462,1.32462,1.32462,1.32462,2.40
897526,2024-01-01 23:56:00,1.32463,1.32465,1.32441,1.32443,43.27
897527,2024-01-01 23:57:00,1.32445,1.32450,1.32435,1.32436,23.85
897528,2024-01-01 23:58:00,1.32435,1.32436,1.32435,1.32435,9.60


In [18]:
train_start = '2021-08-01'
train_end = '2021-09-01'

test_start = '2022-01-01'
test_end = '2022-04-01'

train_data = df[(df.Date > train_start) & (df.Date < train_end)]
test_data = df[(df.Date > test_start) & (df.Date < test_end)]

train_data.loc[:, 'Date'] = pd.to_datetime(train_data.Date)
test_data.loc[:, 'Date'] = pd.to_datetime(test_data.Date)
df.loc[:, 'Date'] = pd.to_datetime(df.Date)

train_data = train_data.set_index('Date')
test_data = test_data.set_index('Date')

df = df.set_index('Date')


In [22]:
train_data['vwap'] = pandas_ta.vwap(high=train_data['High'], low=train_data['Low'], close=train_data['Close'], volume=train_data['Volume'])

# RSI ADX EMA Strategy

In [7]:
def pivot_points(prices, window):
    # Identificar pivotes altos y bajos por separado
    pivots_high = prices.rolling(window=window).apply(lambda x: x.argmax() == len(x) // 2, raw=True)
    pivots_low = prices.rolling(window=window).apply(lambda x: x.argmin() == len(x) // 2, raw=True)
    
    pivots_high = prices[pivots_high == 1]
    pivots_low = prices[pivots_low == 1]
    
    return pivots_high, pivots_low

def count_touches_optimized(prices, pivots, tolerance, lookback_window):
    touches = pd.Series(0, index=prices.index)
    
    for pivot_index, pivot_value in pivots.items():
        window_df = prices.loc[pivot_index:].iloc[:lookback_window]
        valid_touches = (window_df.between(pivot_value - tolerance, pivot_value + tolerance)).sum()
        touches.loc[pivot_index] = valid_touches
    
    return touches

def support_resistance_indicator(prices, window=5, min_touches=3, tolerance=0.01, lookback_window=20):
    # Obtener pivotes altos (resistencias) y bajos (soportes)
    prices = pd.Series(prices)
    pivots_high, pivots_low = pivot_points(prices, window)
    
    # Contar los toques para resistencias y soportes por separado
    touches_high = count_touches_optimized(prices, pivots_high, tolerance, lookback_window)
    touches_low = count_touches_optimized(prices, pivots_low, tolerance, lookback_window)
    
    # Filtrar resistencias y soportes significativos según el número mínimo de toques
    significant_resistances = pivots_high[touches_high >= min_touches].reindex(prices.index).fillna(method='ffill')
    significant_supports = pivots_low[touches_low >= min_touches].reindex(prices.index).fillna(method='ffill')

    return significant_resistances, significant_supports

## Test Entry: Monkey Exit

In [26]:
def vwap(close):
    return train_data.vwap

In [29]:

class RsiAdxEma(Strategy):
    pip_size = 0.0001
    sl_pips = 10
    rr = 1.5
    risk = 1

    adx_period = 5
    rsi_period = 3

    def init(self):
        self.adx = self.I(ta.ADX, self.data.High, self.data.Low, self.data.Close, timeperiod=self.adx_period)
        self.ema_50 = self.I(ta.EMA, self.data.Close, timeperiod=50) 
        self.rsi = self.I(ta.RSI, self.data.Close, self.rsi_period)
        self.engulfing = self.I(ta.CDLENGULFING, self.data.Open, self.data.High, self.data.Low, self.data.Close)
        self.vwap = self.I(vwap, self.data.Close)
        self.random_time = None

    def next(self):
        close_prices = self.data.Close
        actual_close = close_prices[-1]
       
        if self.position:
            first_trade = self.trades[0]
            today = self.data.index[-1].tz_localize('UTC').tz_convert('UTC')
            time_in_position = (today - first_trade.entry_time.tz_localize('UTC').tz_convert('UTC'))
            time_in_position = time_in_position.total_seconds() // 60


            if time_in_position >= self.random_time:
                self.position.close()
                self.random_time = None

        else: 
            rsi_now = self.rsi[-1]
            rsi_anterior = self.rsi[-2]

            if  actual_close > self.ema_50 and rsi_anterior < 20 and rsi_now > 20 and self.adx > 30 and self.engulfing == 100 and actual_close <= self.vwap:
                
                account_risk = self.equity * (self.risk / 100)
                units = round(account_risk / (self.pip_size * self.sl_pips))

                self.buy(size=units)
                self.random_time = random.randint(2, 10)

            elif  actual_close < self.ema_50 and rsi_anterior > 80 and rsi_now < 80 and self.adx > 30 and self.engulfing == -100 and actual_close >= self.vwap:
                account_risk = self.equity * (self.risk / 100)
                
                units = round(account_risk / (self.pip_size * self.sl_pips))

                self.sell(size=units)
                self.random_time = random.randint(2, 10)


bt_train = Backtest(
    train_data, 
    RsiAdxEma, 
    cash=15_000, 
    margin=1/30
)

stats = bt_train.run()

bt_train.plot(filename='./RsiBBands.html', resample=False)

stats


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2021-08-02 00:00:00
End                       2021-08-31 23:59:00
Duration                     29 days 23:59:00
Exposure Time [%]                    0.516175
Equity Final [$]                  14917.92713
Equity Peak [$]                   15090.79454
Return [%]                          -0.547152
Buy & Hold Return [%]                1.157001
Return (Ann.) [%]                   -5.637106
Volatility (Ann.) [%]                3.960851
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -1.273435
Avg. Drawdown [%]                   -0.526749
Max. Drawdown Duration       13 days 04:04:00
Avg. Drawdown Duration        4 days 04:01:00
# Trades                                   20
Win Rate [%]                             50.0
Best Trade [%]                       0.044551
Worst Trade [%]                     -0.044357
Avg. Trade [%]                    

## Test Exit: Monkey Entry

In [9]:

trade_frecuency = stats['# Trades'] / train_data.shape[0]
long_frecuency = stats._trades[stats._trades['Size'] > 0].shape[0] / stats['# Trades']
short_frecuency = stats._trades[stats._trades['Size'] < 0].shape[0] / stats['# Trades']

def random_boolean(prob_true=0.5):
    return random.choices([True, False], weights=[prob_true, 1 - prob_true], k=1)[0]


class RsiAdxEma(Strategy):
    pip_size = 0.0001
    sl_pips = 15
    rr = 1.5
    risk = 1

    adx_period = 5
    rsi_period = 3

    def init(self):
        self.adx = self.I(ta.ADX, self.data.High, self.data.Low, self.data.Close, timeperiod=self.adx_period)
        self.ema_50 = self.I(ta.EMA, self.data.Close, timeperiod=50) 
        self.ema_200 = self.I(ta.EMA, self.data.Close, timeperiod=200) 
        self.rsi = self.I(ta.RSI, self.data.Close, self.rsi_period)
        self.engulfing = self.I(ta.CDLENGULFING, self.data.Open, self.data.High, self.data.Low, self.data.Close)

        self.random = None

    def next(self):
        close_prices = self.data.Close
        actual_close = close_prices[-1]
       
        if self.position:
            if self.position.is_long and crossover(self.ema_50, self.data.Close):
                self.position.close()
                
            elif self.position.is_short and crossover(self.data.Close, self.ema_50):
                self.position.close()

        else: 
            trade = random_boolean(prob_true=trade_frecuency)

            if trade:
                long = random_boolean(prob_true=long_frecuency)

                if long:
                    sl = actual_close - self.sl_pips * self.pip_size
                    account_risk = self.equity * (self.risk / 100)
                    units = round(account_risk / (self.pip_size * self.sl_pips))
                    self.buy(sl=sl, size=units)

                else:
                    sl = actual_close + self.sl_pips * self.pip_size
                    account_risk = self.equity * (self.risk / 100)
                    
                    # Calculate lot size in units
                    units = round(account_risk / (self.pip_size * self.sl_pips))
                    self.sell(sl=sl, size=units)


bt_train = Backtest(
    train_data, 
    RsiAdxEma, 
    cash=15_000, 
    margin=1/30
)

stats = bt_train.run()

# bt_train.plot(filename='./RsiBBands.html', resample=False)

stats


Start                     2021-08-02 00:00:00
End                       2021-08-31 23:59:00
Duration                     29 days 23:59:00
Exposure Time [%]                    6.649354
Equity Final [$]                  14614.73846
Equity Peak [$]                   15474.97783
Return [%]                           -2.56841
Buy & Hold Return [%]                1.157001
Return (Ann.) [%]                  -38.457985
Volatility (Ann.) [%]                6.541844
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -5.558905
Avg. Drawdown [%]                   -0.508152
Max. Drawdown Duration       29 days 07:50:00
Avg. Drawdown Duration        1 days 15:46:00
# Trades                                   83
Win Rate [%]                        31.325301
Best Trade [%]                       0.237358
Worst Trade [%]                     -0.119694
Avg. Trade [%]                    

In [ ]:
bt_test = Backtest(
    test_data, 
    RsiAdxEma, 
    cash=15_000, 
    commission=0.0002,
    margin=1/30
)
 
test_stats = bt_test.run(
    adx_period=stats._strategy.adx_period,
    rsi_period=stats._strategy.rsi_period,
)



test_stats

In [ ]:
bt_test.plot(filename='./ConsecutiveCandlesGridTest.html', resample=False)


# WFO

In [ ]:
def walk_forward(
        strategy,
        data_full,
        warmup_bars,
        lookback_bars=28*1440,
        validation_bars=7*1440,
        cash=15_000, 
        commission=0.0002,
        margin=1/30

):

    stats_master = []
    equity_final = None

    for i in range(lookback_bars, len(data_full)-validation_bars, validation_bars):

        # print(i)

        # To do anchored walk-forward, just set the first slice here to 0
        train_data = data_full.iloc[i-lookback_bars: i]

        print(f'train from {train_data.index[0]} to {train_data.index[-1]}')

        bt_training = Backtest(
            train_data, 
            strategy, 
            cash=cash, 
            commission=commission, 
            margin=margin
        )

        stats_training = bt_training.optimize(
                adx_period=[3, 4, 5],
                rsi_period=[3, 4, 5],
                # sl_pips=[5, 8, 10],
                # rr=[1, 1.5, 2],
                maximize='Return [%]'
        )

        
        validation_data = data_full.iloc[i-warmup_bars:i+validation_bars]
        print(f'validate from {validation_data.index[0]} to {validation_data.index[-1]}')


        bt_validation = Backtest(
            validation_data, 
            strategy, 
            cash=cash if equity_final == None else equity_final, 
            commission=commission, 
            margin=margin
        )

        stats_validation = bt_validation.run(
                adx_period = stats_training._strategy.adx_period,
                rsi_period = stats_training._strategy.rsi_period,
                # sl_pips = stats_training._strategy.sl_pips,
                # rr = stats_training._strategy.rr,
        )
        
        equity_final = stats_validation['Equity Final [$]']
        print(f'equity final: {equity_final}')

        stats_master.append(stats_validation)

    return stats_master

In [ ]:
import pickle


lookback_bars = 60*1440
validation_bars = 7*1440
warmup_bars = 14*60

if os.path.exists("stats.pickle"):
    with open("stats.pickle", "rb") as f:
        stats = pickle.load(f)
else:
    stats = walk_forward(
        RsiAdxEma, 
        test_data, 
        warmup_bars=0, 
        commission=0.0002, 
        margin=1/30, 
        cash=15_000
    )
    
    # with open("stats.pickle", "wb") as f:
    #     pickle.dump(stats, f)

In [ ]:
stats = pd.DataFrame(stats)
stats['Win Rate [%]'].mean()

In [ ]:
plot_stats(df, stats[2], RsiAdxEma, plot=False)

In [ ]:
plot_full_equity_curve(df, stats, warmup_bars = warmup_bars,
                       lookback_bars = lookback_bars, overlay_price = True)

In [ ]:
trades = test_stats._trades.groupby(by=['ExitTime']).agg({'PnL':['sum','count'], 'Duration':'max'})
trades.columns = trades.columns.droplevel(0)
trades = trades.reset_index().rename(columns={'count':'ammount_trades'})
trades = trades.rename(columns={'sum':'profit'})
trades = trades.rename(columns={'max':'minutes_in_trade'})
trades

In [ ]:
trades.minutes_in_trade.describe()
